## Reprocess NIRCam Grism TSO data with new wavelength calibration reference file

Created by B. Hilbert, 26 July 2023.

This notebook downloads an example NIRCam grism time series file and runs the stage 2 and 3 calibration pipelines on the file. This is the procedure you would follow to reprocess your data if new wavelength calibration reference files become available. (The wavelength calibration reference files are called "specwcs" in CRDS.

Following this example, you can reprocess your time series data when new specwcs reference files become available in CRDS.

<span style="color:red">Note that when new reference files become available in CRDS, all data to which those reference files apply will generally be reprocessed using the new reference files within several weeks, and the products in MAST will be updated. But if you wish to reprocess the data yourself, this notebook outlines the method to do that.</span>

Define function to download a named file via the MAST API. The function includes authentication logic, but this example uses public data, so no MAST API token is required.

In [ ]:
from glob import glob
import requests

In [ ]:
from jwst.pipeline.calwebb_spec2 import Spec2Pipeline
from jwst.pipeline.calwebb_tso3 import Tso3Pipeline

In [ ]:
def get_jwst_file(name, mast_api_token=None):
    """Retrieve a JWST data file from MAST archive."""
    mast_url = "https://mast.stsci.edu/api/v0.1/Download/file"
    params = dict(uri=f"mast:JWST/product/{name}")
    if mast_api_token:
        headers = dict(Authorization=f"token {get_mast_api_token()}")
    else:
        headers = {}
    r = requests.get(mast_url, params=params, headers=headers, stream=True)
    r.raise_for_status()
    with open(name, "wb") as fobj:
        for chunk in r.iter_content(chunk_size=1024000):
            fobj.write(chunk)

List of filenames for a program that contains an example Grim TSO observation.

In [ ]:
example_tso_file = 'jw01068007001_03103_00001-seg001_nrcalong_rateints.fits'
level2_asn_file = 'jw01068-o007_20230626t065143_tso-spec2_00001_asn.json'
level3_asn_file = 'jw01068-o007_20230626t065143_tso3_00001_asn.json'
files = [example_tso_file, level2_asn_file, level3_asn_file]

Download example files from MAST

In [ ]:
for file in files:
    print(f'Downloading {file}')
    get_jwst_file(file)

Confirm that the files were downloaded

In [ ]:
sorted(glob('jw01068*'))

Run the stage 2 pipeline. The new specwcs files will be automatically downloaded and used in the assign_wcs step. The name of this specwcs file is be given in the line below that contains the string: "Prefetch for SPECWCS reference file".

We run the pipeline using the stage 2 association file, which lists the rateints file within it.

In [ ]:
level2 = Spec2Pipeline.call(level2_asn_file, save_results=True)

The stage 2 pipeline saved the new calints and x1dints files

In [ ]:
sorted(glob('jw01068*'))

Run the stage 3 pipeline. The input to the pipeline is the stage 3 association file, which lists the calints file within it.

In [ ]:
level3 = Tso3Pipeline.call(level3_asn_file, save_results=True)

The stage 3 pipeline saved updated versions of the crfints, median, x1dints, and whtlt files.

In [ ]:
sorted(glob('jw01068*'))